# Quick Start — Run FarmFederate on Colab 🚀

Follow these steps in order (copy the provided cells into Colab):

1. **Pre-setup**: Run the *pre-setup* cell to install missing packages and optionally upload `kaggle.json`. This sets safe defaults (DRY_RUN=1).
2. **Dry-run**: The pre-setup cell will run a dry-run of `FarmFederate_Kaggle_Complete.py` to validate config and write manifests (`results/dataset_discovery_manifest.json`, `datasets_report.json`). Inspect these files.
3. **Review & Accept**: If using Kaggle competitions (e.g., Plant Pathology), accept the terms on Kaggle and ensure `kaggle.json`/env vars are set. Set `HF_TOKEN` when prompted if you need HF datasets.
4. **Full Run**: If dry-run looks good, proceed with the full run when prompted by the cell (it will set DRY_RUN=0). You can opt to clone GitHub fallback repos (CLONE_GITHUB_REPOS=1).
5. **Drive Sync (optional)**: When you mount Drive, the one-click cell can copy `results/` and `checkpoints/` to your Drive folder for persistence.
6. **Inspect Outputs**: After the run check `results/` (JSONs, metrics, plots) and `plots/` for images. Use `results/dataset_discovery_manifest.json` to review dataset acquisition details.

Notes: Start with the dry-run to avoid long downloads by accident. If you want, run the **One-click** cell to perform the whole workflow interactively.


# FarmFederate: Colab Runbook

This notebook prepares a Google Colab environment and runs the project script safely. Follow the cells in order and set the required credentials (Kaggle and HuggingFace) before enabling full downloads and training. The notebook is structured with safety guards so it will not perform heavy downloads unless you set RUN_ON_COLAB=1.

*Outline:* Colab setup → Install dependencies → Credentials upload → Flags & guards → Run discovery/downloads (download-only or full) → Training / Federated runs (fast-mode) → Plots and export.


In [ ]:
# 1) Colab & Environment Setup

# Mount Google Drive (optional) and check GPU runtime
from google.colab import drive
print('Mounting Google Drive... (optional)')
drive.mount('/content/drive')

# Check GPU
import torch
print('Device:', torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
print('CUDA available:', torch.cuda.is_available())

# Safety: enable heavy runs explicitly
import os
print('IN_COLAB detected. To enable downloads and heavy runs set environment variable RUN_ON_COLAB=1')
print('Example: %env RUN_ON_COLAB 1')


In [ ]:
# 2) Install Dependencies (run this cell)
# Installs commonly required packages. Runs quietly in Colab.
!pip install -q transformers datasets peft torch torchvision scikit-learn kaggle imgaug

print('Installed core dependencies.')

In [ ]:
# 3) Notebook Flags & Runtime Guards
import os
# Example toggles
os.environ.setdefault('RUN_ON_COLAB', '0')  # Set to '1' to allow heavy ops
os.environ.setdefault('DRY_RUN', '1')       # Set to '0' to enable full downloads
os.environ.setdefault('DOWNLOAD_ONLY', '0')
os.environ.setdefault('REPORT_ONLY', '0')

print('RUN_ON_COLAB=', os.environ['RUN_ON_COLAB'])
print('DRY_RUN=', os.environ['DRY_RUN'])
print('DOWNLOAD_ONLY=', os.environ['DOWNLOAD_ONLY'])
print('REPORT_ONLY=', os.environ['REPORT_ONLY'])

print('\nTo enable full Colab run, execute:')
print('%env RUN_ON_COLAB 1')
print('%env DRY_RUN 0')


In [ ]:
# 4) Kaggle & HuggingFace Credentials
from google.colab import files
import os
print('Upload kaggle.json (optional) or set KAGGLE_USERNAME and KAGGLE_KEY as environment variables')
print('If you have kaggle.json file, use the upload widget:')

uploaded = files.upload()
if uploaded:
    kaggle_dir = '/root/.kaggle'
    os.makedirs(kaggle_dir, exist_ok=True)
    for fn in uploaded.keys():
        dest = os.path.join(kaggle_dir, 'kaggle.json')
        open(dest, 'wb').write(uploaded[fn])
        try:
            os.chmod(dest, 0o600)
        except Exception:
            pass
    print('Saved uploaded kaggle.json to ~/.kaggle/kaggle.json')

print('\nTo set HF_TOKEN securely, run:')
print('%env HF_TOKEN your_hf_token_here')


In [ ]:
# 5) Quick validation & small DRY_RUN test (safe)
# This will run the script in dry-run mode to validate flags and manifest writing without downloads.
import os
os.environ['DRY_RUN'] = os.environ.get('DRY_RUN', '1')
print('Running a quick dry-run of dataset discovery (DRY_RUN=1) to validate configuration...')
!python FarmFederate_Kaggle_Complete.py --dry-run


In [ ]:
# 6) Full run (enable when ready)
# Set RUN_ON_COLAB=1 and toggle other flags as needed before running this cell.
import os
os.environ['RUN_ON_COLAB'] = '1'  # enable downloads / cloning
os.environ['DRY_RUN'] = '0'       # disable dry-run to allow downloads
# Default to cloning fallback sources
os.environ['CLONE_GITHUB_REPOS'] = '1'
print('RUN_ON_COLAB, DRY_RUN, CLONE_GITHUB_REPOS set to:', os.environ['RUN_ON_COLAB'], os.environ['DRY_RUN'], os.environ['CLONE_GITHUB_REPOS'])

# Run the main script (this performs discovery, optional downloads, dataset building)
# It may take a long time depending on datasets and whether you allow Kaggle/HTTP downloads.
!python FarmFederate_Kaggle_Complete.py

# Notes & Checklist

- ☑️ Upload `kaggle.json` or set `KAGGLE_USERNAME` and `KAGGLE_KEY` via `%env`.
- ☑️ Set `HF_TOKEN` via `%env HF_TOKEN <token>` for HuggingFace access if needed.
- ⚠️ Make sure you accept Kaggle competition terms for `plant-pathology-2020-fgvc7` if using that dataset.
- 🔧 If anything fails, start with the quick DRY_RUN cell and inspect `results/dataset_discovery_manifest.json` and `datasets_report.json`.

If you'd like, I can: (a) push this notebook to the repo and (b) prepare a short Colab-specific README cell with step-by-step commands for one-click runs. Which would you prefer next?

In [ ]:
### One-click: Full setup + run (copy & paste into Colab)

# This single cell installs missing packages, mounts Drive (optional), uploads credentials,
# runs a DRY-RUN to validate, then optionally proceeds to the full run and syncs results to Drive.

import importlib, subprocess, sys, os, getpass
from google.colab import drive, files

# Install missing packages
packages = ['transformers','datasets','peft','torch','torchvision','scikit-learn','kaggle','imgaug']
toinstall = []
for pkg in packages:
    try:
        importlib.import_module(pkg if pkg != 'scikit-learn' else 'sklearn')
    except Exception:
        toinstall.append(pkg)
if toinstall:
    print('Installing:', toinstall)
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q'] + toinstall)
else:
    print('All required packages appear to be installed.')

# Optional: mount Drive
use_drive = input('Mount Google Drive to save results? (y/N): ').strip().lower() == 'y'
drive_path = None
if use_drive:
    drive.mount('/content/drive')
    drive_path = input('Enter Drive folder path to save results (e.g., /content/drive/MyDrive/FarmFederate-results) or leave blank to use default: ').strip()
    if not drive_path:
        drive_path = '/content/drive/MyDrive/FarmFederate-results'

# Upload kaggle.json optionally
print('\nUpload kaggle.json now (or press cancel to set KAGGLE_USERNAME/KAGGLE_KEY env vars manually)')
uploaded = files.upload()
if uploaded:
    kaggle_dir = '/root/.kaggle'
    os.makedirs(kaggle_dir, exist_ok=True)
    for fn, data in uploaded.items():
        open(os.path.join(kaggle_dir, 'kaggle.json'), 'wb').write(data)
    try:
        os.chmod(os.path.join(kaggle_dir, 'kaggle.json'), 0o600)
    except Exception:
        pass
    print('Saved kaggle.json to ~/.kaggle/kaggle.json')

# Set flags and HF_TOKEN
os.environ['RUN_ON_COLAB'] = '1'
os.environ['DRY_RUN'] = '1'  # start with dry-run
os.environ['CLONE_GITHUB_REPOS'] = '1'  # default to cloning fallback repos during discovery
hf = getpass.getpass('Enter HF_TOKEN (leave blank to skip): ')
if hf:
    os.environ['HF_TOKEN'] = hf

print('\nRunning DRY-RUN validation (this will not download data) ...')
!python FarmFederate_Kaggle_Complete.py --dry-run

# Prompt for full run
resp = input('\nDRY-RUN complete. Proceed to full run and allow downloads/clones? (y/N): ').strip().lower()
if resp == 'y':
    clone_resp = input('Enable cloning of GitHub fallback repos during discovery? (y/N): ').strip().lower()
    if clone_resp != 'y':
        os.environ['CLONE_GITHUB_REPOS'] = '0'
    os.environ['DRY_RUN'] = '0'
    print('\nStarting full run (this may take a long time). Logs will appear below...')
    !python FarmFederate_Kaggle_Complete.py
    # Optionally copy results to Drive
    if drive_path:
        try:
            import shutil
            os.makedirs(drive_path, exist_ok=True)
            print('Copying results/ and checkpoints/ to', drive_path)
            shutil.copytree('results', os.path.join(drive_path, 'results'), dirs_exist_ok=True)
            if os.path.exists('checkpoints'):
                shutil.copytree('checkpoints', os.path.join(drive_path, 'checkpoints'), dirs_exist_ok=True)
            print('Copied artifacts to Drive.')
        except Exception as e:
            print('Failed to copy to Drive:', e)
else:
    print('Exiting. Re-run the full run when you are ready (set DRY_RUN=0 to run fully).')